In [1]:
from implementations import  *
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
import seaborn as sns
plt.rcParams['text.usetex'] = False
import math
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

### Loading Data

In [3]:
x_train, x_test, y_train, train_ids, test_ids = load_csv_data("data/dataset_to_release/", sub_sample=False) 

In [4]:
y_train = (y_train + 1) / 2

In [12]:
#get column names
column_names = np.genfromtxt("data/dataset/x_train.csv", delimiter=",", dtype=str, max_rows=1)
column_names.shape
column_names = column_names[1:]

#Filter NaNs
nan_counts = np.isnan(x_train).sum(axis=0)
columns_to_remove = np.where(nan_counts > x_train.shape[0]/3)[0]
column_names_clean = np.delete(column_names, columns_to_remove, axis=0)
 
x_clean = np.delete(x_train, columns_to_remove, axis=1)
x_clean_test = np.delete(x_test, columns_to_remove, axis=1)


x_clean[np.isnan(x_clean)] = np.take(np.nanmedian(x_clean), np.isnan(x_clean).nonzero()[1])
x_clean_test[np.isnan(x_clean_test)] = np.take(np.nanmedian(x_clean_test), np.isnan(x_clean_test).nonzero()[1])

IndexError: index 70 is out of bounds for axis 0 with size 1

In [6]:
relevant_features=['GENHLTH','_AGEG5YR','BPHIGH4'
,'_RFHLTH','_RFHYPE5','_DRDXAR1','DIABETE3'
,'DIFFWALK','TOLDHI2','_RFCHOL','HAVARTH3','QLACTLM2'
,'CVDSTRK3','PNEUVAC3'
,'CHCCOPD1','SMOKE100','SEX','DIFFALON','_LMTACT1','CHCOCNCR','_BMI5CAT'
,'DIFFDRES']

In [10]:
column_names[columns_to_remove]

array(['PVTRESD1', 'COLGHOUS', 'STATERES', 'CELLFON3', 'LADULT',
       'NUMADULT', 'NUMMEN', 'NUMWOMEN', 'CTELNUM1', 'CELLFON2', 'CADULT',
       'PVTRESD2', 'CCLGHOUS', 'CSTATE', 'LANDLINE', 'HHADULT', 'GENHLTH',
       'HLTHPLN1', 'BLOODCHO', 'CHCSCNCR', 'SEX', 'NUMPHON2', 'CPDEMO1',
       'VETERAN3', 'QLACTLM2', 'STOPSMK2', 'LASTSMK2', 'USENOW3',
       'DRNK3GE5', 'MAXDRNKS', 'FRUITJU1', 'EXRACT21', 'EXEROFT2',
       'EXERHMM2', 'STRENGTH', 'ARTHDIS2', 'ARTHSOCL', 'JOINPAIN',
       'SEATBELT', 'IMFVPLAC', 'PNEUVAC3', 'WHRTST10', 'PDIABTST',
       'PREDIAB1', 'INSULIN', 'BLDSUGAR', 'FEETCHK2', 'DOCTDIAB',
       'CHKHEMO3', 'FEETCHK', 'EYEEXAM', 'DIABEYE', 'DIABEDU', 'CAREGIV1',
       'CRGVREL1', 'CRGVLNG1', 'CRGVHRS1', 'CRGVPRB1', 'CRGVPERS',
       'CRGVHOUS', 'CRGVMST2', 'CRGVEXPT', 'VIDFCLT2', 'VIREDIF3',
       'VIPRFVS2', 'VINOCRE2', 'VIEYEXM2', 'VIINSUR2', 'VICTRCT4',
       'VIGLUMA2', 'VIMACDG2', 'CIMEMLOS', 'CDHOUSE', 'CDASSIST',
       'CDHELP', 'CDSOCIAL', 'CDDISCU

In [7]:
#expand matrix by one hot encoding
x_train_one_hot = one_hot_encoder(x_clean, column_names_clean, relevant_features)
x_test_one_hot = one_hot_encoder(x_clean_test, column_names_clean, relevant_features)

ValueError: 'GENHLTH' is not in list

In [ ]:
ratio = 0.8
seed = 1
x_train_split, x_val_split, y_train_split, y_val_split = train_test_split(x_train_one_hot, y_train, test_size=1-ratio, random_state=seed)

In [ ]:
print(y_train_split)
print(y_val_split)

[0. 0. 0. ... 0. 0. 0.]
[1. 0. 1. ... 0. 0. 0.]


In [ ]:
# Appy SMOTE on the training data for oversampling
x_train_smote, y_train_smote = SMOTE(x_train_split, y_train_split, k=5, ratio=0.5)
print("oversampling over")

11567


Current iteration=0
Current iteration=100
Current iteration=200
Current iteration=300
Current iteration=400
Current iteration=500
Current iteration=600
Current iteration=700
Current iteration=800
Current iteration=900
Current iteration=1000
Current iteration=1100
Current iteration=1200
Current iteration=1300
Current iteration=1400
Current iteration=1500
Current iteration=1600
Current iteration=1700
Current iteration=1800
Current iteration=1900
Current iteration=2000
Current iteration=2100
Current iteration=2200
Current iteration=2300
Current iteration=2400
Current iteration=2500
Current iteration=2600
Current iteration=2700
Current iteration=2800
Current iteration=2900
Current iteration=3000
Current iteration=3100
Current iteration=3200
Current iteration=3300
Current iteration=3400
Current iteration=3500
Current iteration=3600
Current iteration=3700
Current iteration=3800
Current iteration=3900
Current iteration=4000
Current iteration=4100
Current iteration=4200
Current iteration=4300


In [ ]:
# Undersampling the training data
x_train_under, y_train_under = undersample_majority(x_train_smote, y_train_smote, ratio=0.8)
print("undersampling over")

(239373, 98)
undersampling over


In [ ]:
x_stan_train, mean_x, std_x = standardize(x_train_under)
x_stan_val = (x_val_split-mean_x) / std_x
x_stan_test = (x_test_one_hot - mean_x) / std_x

In [ ]:
y, tx_train = build_model_data(y_train_under, x_stan_train)
initial_w = np.random.rand(tx_train.shape[1])
w,loss = reg_logistic_regression(y, tx_train, 0.5, initial_w , 10000, 0.001)

Initial loss=1.8494772477563646
Current iteration=0, loss=19.565183798739692
Current iteration=100, loss=15.970274943353518
Current iteration=200, loss=13.054159582150875
Current iteration=300, loss=10.690511190101848
Current iteration=400, loss=8.77507620624705
Current iteration=500, loss=7.222809011855585
Current iteration=600, loss=5.964793989787837
Current iteration=700, loss=4.945023131010681
Current iteration=800, loss=4.117980933973116
Current iteration=900, loss=3.447138930297581
Current iteration=1000, loss=2.9031856751508265
Current iteration=1100, loss=2.462121070054537
Current iteration=1200, loss=2.1044796350515735
Current iteration=1300, loss=1.8143478478707395
Current iteration=1400, loss=1.5789000126542891
Current iteration=1500, loss=1.3874638418478435
Current iteration=1600, loss=1.2321863871271603
Current iteration=1700, loss=1.1063234760019423
Current iteration=1800, loss=1.00434295705826
Current iteration=1900, loss=0.9217470517297166
Current iteration=2000, loss=0

In [ ]:
val_tx = np.c_[np.ones(x_stan_val.shape[0]), x_stan_val]
y_pred_val = predict(val_tx, w, threshold = 0.5)
y_pred_val = (y_pred_val + 1) / 2
print("Accuracy for Validation Set:", accuracy(y_val_split, y_pred_val))
print("F1 score for Validation Set:", compute_f1_score(y_val_split, y_pred_val))


TypeError: predict() got an unexpected keyword argument 'threshold'

In [ ]:
test_tx = np.c_[np.ones(x_stan_test.shape[0]), x_stan_test]
y_pred = predict(test_tx,w)
create_csv_submission(test_ids, y_pred, 'submission.csv')